# All Basins

In [1]:
import xarray as xr
from matplotlib.pylab import *
import numpy as np
import pandas as pd
from geopy import distance
from mpl_toolkits.axes_grid1 import make_axes_locatable
from geographiclib.geodesic import Geodesic
import time

def get_bearing(lat1, lat2, long1, long2):
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    return brng

In [2]:
xds_tracks = xr.open_dataset('Allstorms.ibtracs_wmo.v03r09.nc')

In [9]:
import datetime

d = datetime.date(1997, 12, 31)
np.datetime64(d)

numpy.datetime64('1997-12-31')

In [13]:
lats = []
lons = []
pres = []
time = []
basin = []
dist2land = []
st_idx = []
for i in xds_tracks.storm.values:
    idx = np.argwhere(xds_tracks.sel(storm = i).time_wmo[~np.isnan(xds_tracks.sel(storm = i).time_wmo)].values > np.datetime64(d))
   #idx me selecciona los datos para fechas a partir de 1998
    if len(idx) == 0: #esta condicion significa que es un array vacio, con lo cual no hay fechas a partir de 1998
        continue
    else:
        
        lats.append(xds_tracks.sel(storm = i).lat_wmo.values)
        lons.append(xds_tracks.sel(storm = i).lon_wmo.values)
        pres.append(xds_tracks.sel(storm = i).pres_wmo.values)
        time.append(xds_tracks.sel(storm = i).time_wmo.values)
        basin.append(xds_tracks.sel(storm = i).basin.values)
        dist2land.append(xds_tracks.sel(storm = i).dist2land.values)
        st_idx.append(i)

In [15]:
All_basin = xr.Dataset({'latitud': (('storm','time'), np.array(lats)),
           'longitud':(('storm','time'), np.array(lons)),
           'presion': (('storm','time'), np.array(pres)),
           'cuenca': (('storm','time'), np.array(basin)),
           'distancia_tierra': (('storm','time'), np.array(dist2land)),
           'fechas': (('storm','time'), np.array(time))},
          {'storm': np.array(st_idx),
          'time': xds_tracks.time.values})

In [16]:
All_basin

<xarray.Dataset>
Dimensions:           (storm: 1636, time: 137)
Coordinates:
  * storm             (storm) int64 5630 5632 5633 5634 ... 7263 7264 7265 7266
  * time              (time) int64 0 1 2 3 4 5 6 ... 130 131 132 133 134 135 136
Data variables:
    latitud           (storm, time) float32 -12.3 -12.4 -12.5 ... nan nan nan
    longitud          (storm, time) float32 173.5 172.9 172.5 ... nan nan nan
    presion           (storm, time) float32 997.0 987.0 985.0 ... nan nan nan
    cuenca            (storm, time) float32 4.0 4.0 4.0 4.0 ... nan nan nan nan
    distancia_tierra  (storm, time) float32 723.0 691.0 647.0 ... nan nan nan
    fechas            (storm, time) datetime64[ns] 1998-01-03T00:00:00.000043520 ... NaT

In [17]:
All_basin.to_netcdf('All_basin.nc4')